In [6]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Literal, Optional, List
import logging
import asyncio
import os
from dotenv import load_dotenv
from pathlib import Path
load_dotenv(Path("../.env"))
logger = logging.getLogger(__name__)

In [2]:
class OpenAIClient:
    def __init__(self, api_key: str) -> None:
        """
        Args:
            model_name: The name of the openai model we are using
            api_key: The api key for our openai model
        Returns:
        """
        self.client = OpenAI(api_key=api_key)

    def get_client(self) -> OpenAI:
        """
        Args:
            None

        Returns:
            The openai client
        """
        return self.client

In [3]:
llm = OpenAIClient(api_key=os.getenv("OPENAI_API_KEY")).get_client()

In [4]:
from typing import Any, Dict, Optional, Union
from fastmcp.client.client import Client
from contextlib import asynccontextmanager


class MCPClient:
    def __init__(self, config: Union[str, dict] = "http://localhost:8050/sse"):
        """Initialize the MCP client.

        Args:
            config (Union[str, dict]): Either a URL string or a configuration dictionary.
                If string: Treated as the URL of the MCP server.
                If dict: Should follow the MCP configuration format with 'mcpServers' key.
        """
        self.config = config
        self._client = None
        self._is_connected = False

    async def connect(self):
        """Connect to the MCP server(s)."""
        if self._is_connected:
            return

        if isinstance(self.config, str):
            # For SSE transport, we just need the URL
            self._client = Client(self.config)
        else:
            # Configuration mode with multiple servers
            self._client = Client(self.config)

        await self._client.__aenter__()
        self._is_connected = True

    async def disconnect(self):
        """Disconnect from the MCP server(s)."""
        if self._is_connected and self._client:
            await self._client.__aexit__(None, None, None)
            self._is_connected = False
            self._client = None

    @asynccontextmanager
    async def session(self):
        """Context manager for session management."""
        try:
            await self.connect()
            yield self
        finally:
            await self.disconnect()

    async def list_servers(self) -> list:
        """List available MCP servers."""
        if not self._is_connected:
            raise RuntimeError("Not connected to MCP server(s)")
        return list(self._client.servers.keys())

    async def list_tools(self) -> list:
        """List available tools.

        Returns:
            list: List of available tools.
        """
        if not self._is_connected:
            raise RuntimeError("Not connected to MCP server(s)")
        return await self._client.list_tools()

    async def get_tools(self) -> list[dict[str, Any]]:
        """Retrieve tools in a format compatible with OpenAI function calling.

        Returns:
            list[dict[str, Any]]: List of tools in OpenAI function calling format.
        """
        if not self._is_connected:
            raise RuntimeError("Not connected to MCP server(s)")

        tools = await self.list_tools()
        openai_tools = []

        for tool in tools:
            openai_tools.append(
                {
                    "type": "function",
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": {
                        "type": "object",
                        "properties": tool.inputSchema.get("properties", {}),
                        "required": tool.inputSchema.get("required", []),
                    },
                }
            )

        return openai_tools

    async def call_tool(
        self, tool_name: str, arguments: Dict[str, Any], server: Optional[str] = None
    ) -> Any:
        """Call a tool.

        Args:
            tool_name (str): The name of the tool to call.
            arguments (Dict[str, Any]): The arguments to pass to the tool.
            server (str, optional): Specific server to call the tool on.
                                 If None, the client will try to find the tool
                                 in any of the available servers.

        Returns:
            Any: The result of the tool call.
        """
        if not self._is_connected:
            raise RuntimeError("Not connected to MCP server(s)")

        result = await self._client.call_tool(tool_name, arguments, server)
        return result.content[0].text if result.content else None

In [7]:
        mcp_client = MCPClient()
        await mcp_client.connect()

        logger.info("Getting tools from MCP...")
        tools = await mcp_client.get_tools()
        logger.info(f"Loaded {len(tools)} tools from MCP")

In [8]:
tools

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get current temperature for provided coordinates in celsius',
  'parameters': {'type': 'object',
   'properties': {'latitude': {'title': 'latitude', 'type': 'string'},
    'longitude': {'title': 'longitude', 'type': 'string'}},
   'required': ['latitude', 'longitude']}},
 {'type': 'function',
  'name': 'wiki_search',
  'description': 'Get current temperature for provided coordinates in celsius',
  'parameters': {'type': 'object',
   'properties': {'query': {'title': 'Query', 'type': 'string'},
    'sentences': {'default': 2, 'title': 'Sentences', 'type': 'integer'}},
   'required': ['query']}},
 {'type': 'function',
  'name': 'save_txt',
  'description': 'Save text to a .txt file',
  'parameters': {'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'filename': {'default': 'output.txt',
     'title': 'Filename',
     'type': 'string'}},
   'required': ['text']}},
 {'type': 'function',
  'n

In [14]:
# Define prompt for planner agent
PLANNER_AGENT_PROMPT = """
You are an expert essay writer planner.
You take in essay writing request on a given topic or review an essay, and create comprehensive plans, breaking down the main task of writing an essay into smaller actionable tasks.

CORE PRINCIPLE: Be direct and action-oriented. Minimize follow-up questions.

DEFAULT ASSUMPTIONS FOR REQUESTS:
- The request is about writing an essay on a given topic.
- The request might be vague or unclear, one word, or unclear intent
- The request might be very specific or clear
- Request might be to simply review the essay

IMMEDIATE PLANNING APPROACH:
**WORKFLOW:**
1. Always start by creating a plan (for writing an or reviewing an essay) with detailed tasks.
2. Plan should consist of multiple tasks, This is an example but is not limited to [write introduction, research topic, write folow-up, write followup, write conclusion, review, edit, proofread, return essay]
or [review, edit, proof read, return essay]
3. Plan should be specific and actionable
4. For each task in the plan, you MUST assign a tool to perform the task. Fail to do so will result in an task FAIL.
7. YOU must determine how many body paragraphs are sufficient to address the topic.
8. Tools will be given to you and you MUST use them to perform each task with the given description.
FAILURE TO CALL A TOOL WILL RESULT IN A TASK FAIL

TOOL CALLING STRATEGY:
- AVOID repetative tool calls
- Use tools APPROPRIATELY
Example of GOOD tool call 
Task= "research this topic with a this query" -> call research tool
Task ="need to write about a certain topic" -> use results from tool call to write or simply just a query to write
Example of BAD tool call
Task= "Write about a certain topic" -> call research tool
Tool usage MUST make sense with task

MINIMAL QUESTIONS STRATEGY:
- For vauge requests such as single words: generate an interesting topic ie: star wars -> star wars impact on modern culture, then plan and create tasks
- For detailed requests: Create multiple tasks 

You will be given a output format that you must adhere to.

Generate plans immediately without asking follow-up questions unless absolutely necessary.
"""

In [15]:
from typing import Any, Dict, List, Optional, Literal, ForwardRef

class ToolArguments(BaseModel):
    keys: List[str] = Field(description="A list of arguments to a tool")
    values: List[str] = Field(description="A list of argument values to a tool")
    # arguments: Dict[str, Any] = Field(description="A dictionary where keys are tool arguments and values are the tool call values")


class ToolCall(BaseModel):
    """Represents a tool call request from the LLM."""

    id: str = Field(description="The ID of the tool call.")
    name: str = Field(description="The name of the tool to call.")
    arguments: ToolArguments = Field(description="The arguments to call the tool with.")


class ToolCalls(BaseModel):
    id: int = Field(description="An ID for the tool calls")
    tool_calls: List[ToolCall] = Field(
        description="A list of tools to be executed sequentially."
    )



class PlannerTask(BaseModel):
    """Represents a single task generated by the Planner."""

    id: int = Field(description="Sequential ID for the task.")
    description: str = Field(
        description="Clear description of the task to be executed."
    )
    tool_calls: List[ToolCall] = Field(
        description="A list of tools to be executed sequentially to complete the task"
    )
    status: Optional[
        Literal[
            "input_required",
            "completed",
            "error",
            "pending",
            "incomplete",
            "todo",
            "not_started",
        ]
    ] = Field(default="input_required", description="Status of the task")


class Plan(BaseModel):
    """Output schema for the Planner Agent."""

    original_query: str = Field(description="The original user query for context.")
    description: str = Field(description="Clear description of the overall plan.")
    tasks: List[PlannerTask] = Field(
        description="A list of tasks to be executed sequentially."
    )


class ToolResult(BaseModel):
    """Represents the result of a tool execution."""

    tool_call_id: str = Field(description="The ID of the tool call this result is for.")
    result: str = Field(description="The result of the tool execution.")
    is_error: bool = Field(
        default=False, description="Whether the tool execution resulted in an error."
    )


class ResponseFormat(BaseModel):
    """Respond to the user in this format."""

    status: Literal["input_required", "completed", "error"] = "input_required"
    question: str = Field(
        description="Input needed from the user to generate the code search plan"
    )
    content: Plan = Field(
        description="List of tasks when the code search plan is generated"
    )


class TaskExecutionResponse(BaseModel):
    """Represents a single task generated by the Planner."""

    id: int = Field(description="Id of task we are executing.")
    description: str = Field(description="Clear description of the task to be executed.")
    tools_sueggested: str = Field(description="A list of the tools suggested for the task")
    response_type: Optional[
        Literal[
            "tool_calls",
            "text",
        ]
    ] = Field(default="input_required", description="The response type of the task execution")
    tool_calls: List[ToolCall] = Field(description="A list of tool calls to be executed. Empty if response_type is text")


class ContentResponse(BaseModel):
    type: str = Field(
        description="The type of content such as body paragraph, introduction conclusion etc"
    )
    context_used: str = Field(description="The context used to generate the content")
    content: str = Field(description="The content generated by the agent")

In [16]:
class PlannerAgent:
    def __init__(
        self,
        dev_prompt,
        llm,
        messages,
        tools,
        model_name: str = "gpt-4.1-mini",
    ):
        self.model_name = model_name
        self.dev_prompt = dev_prompt
        self.llm = llm
        self.messages = messages
        self.tools = tools
        if self.dev_prompt:
            self.messages.append({"role": "developer", "content": self.dev_prompt})

    def add_messages(self, query: str):
        self.messages.append({"role": "user", "content": query})

    def plan(self, query: str):
        """Create a detailed plan to complete the request of the user.

        Args:
            query (str): The request of the user.

        Returns:
            Plan: The plan to complete the request of the user.
        """
        self.add_messages(query=query)
        response = self.llm.responses.parse(
            model=self.model_name,
            input=self.messages,
            tools=self.tools,
            text_format=Plan,
        )
        return response

In [66]:
class Executor:
    """Executor Class.

    Methods:


    Attributes:


    """

    def __init__(self):
        """
        Initialize the orchestrator
        """
        self.tool_call_history: list = []
        
    def print_task(self, task: PlannerTask, previous_task_results: list) -> None:
        """Print the given task generated by the planner agent

        Args:
            task: The task to print.
            previous_task_results: The results of the previous task.

        Returns:
            None
        """
        print(f"""
              Executing task: \n
              Task ID: {task.id} \n
              Task description: {task.description} \n
              Tool calls: {task.tool_calls} \n
              Task status: {task.status} \n
              Previous Task Results: {previous_task_results}
        """)

    def print_plan(self, plan: Plan) -> None:
        """Print the given plan generated by the planner agent

        Args:
            plan: The plan to print.

        Returns:
            None
        """
        print(f"""
              Plan to execute is: \n
              Plan ID: {plan.original_query}
              Plan Description: {plan.description}
         """)

    async def call_tool(self, tool_calls: list[dict]) -> list[dict]:
        """Receives a list of tool calls and calls the tools

        Args:
            tool_calls: Either a list of tool call dicts or a string error message

        Returns:
            list[dict]: The results of the tool calls or error information
        """
        # If we received an error message instead of tool calls
        if isinstance(tool_calls, str):
            return [{"error": True, "message": tool_calls}]

        # # Ensure tool_calls is a list
        if not isinstance(tool_calls, list):
            return [
                {
                    "error": True,
                    "message": f"Expected list of tool calls, got {type(tool_calls).__name__}",
                }
            ]
        print(f'CALLING TOOLS: {tool_calls}')
        results = []  # Tool call results
        print(f'TOOL_CALLS: {tool_calls}')
        for tool in tool_calls:  # For each tool
            try:  # Try to call the tool
                if not isinstance(tool, dict):  # If tool is not a dict return error
                    results.append(
                        {
                            "error": True,
                            "message": f"Expected dict, got {type(tool).__name__}",
                        }
                    )
                    continue
                # Extract tool name and arguments
                name = tool['name']
                arguments = tool['arguments']
                print(f'NAME: {name}')
                print(f'ARGUMENTS: {arguments}')
                print()
                # If tool name is missing return error
                if not name:
                    results.append(
                        {"error": True, "message": "Tool call missing 'name' field"}
                    )
                    continue

                # Call the tool through MCP client
                result = await mcp_client.call_tool(name, arguments)
                # append tool call reults. Includes name, arguments, and result
                results.append(
                    {"result": result}
                )
                self.tool_call_history.append({"name": name, "arguments": arguments, "result": result, "error": False})
            
            # Handle exceptions
            except Exception as e:
                print("AT EXCEPTION")
                results.append(
                    {
                        "error": True,
                        "name": name if "name" in locals() else "unknown",
                        "message": f"Error calling tool: {str(e)}",
                    }
                )
        print(f'RESULTS: {results}')
        return results
  
    async def execute_task(
        self, task: PlannerTask, previous_task_results: list
    ) -> list[dict]:
        """Execute the given task generated by the planner agent

        Args:
            task: The task to execute.

        Returns:
            A list of results, each containing:
                - name: The name of the tool called.
                - arguments: The arguments passed to the tool.
                - result: The result of the tool call.
                - error: A boolean indicating whether an error occurred.
        """

        # print current task
        self.print_task(task, previous_task_results)
        tool_calls = [] # var to hold tool_calls in current task
        for i in range(len(task.tool_calls)): # for every tool call in the task
            tool_call = task.tool_calls[i] # select the tool call
            tools = self.extract_tools(tool_call) # extract the tool into {name: tool_name, arguments: {...} 
            tool_calls.append(tools) # add tool to tool_Calls list
        
        tool_call_results = await self.call_tool(tool_calls) # call the tools, should return [{'result': result} ...]
        results = [result['result'] for result in tool_call_results]
        
        return results

    def extract_tools(self, tool_call):
        """ """
        tool = {
            'name': tool_call.name.split('.')[-1],
            'arguments': {}
        }
        tool_call_keys = tool_call.arguments.keys
        tool_call_values = tool_call.arguments.values
        for i in range(len(tool_call_values)):
            tool["arguments"][tool_call_keys[i]] = tool_call_values[i]

        return tool

    
    async def execute_plan(self, plan: Plan) -> list:
        """Execute the given plan generated by the planner agent

        Args:
            plan: The plan to execute.

        Returns:
            A list of results
        """
        self.print_plan(plan) # print the plan
        results = ['no context yet'] # list to hold results of each task execution.
        for i in range(len(plan.tasks)): # iterate through tasks
            task: PlannerTask = plan.tasks[i] # select the task
            res = await self.execute_task(task, results) # execute task
            # append task execution results to list
            results.append(
                    {
                        'task': task.description,
                        'results': res,
                    }
            )

        return results

In [67]:
executor = Executor()
logger.info("Successfully initialized Orchestrator")

In [19]:
planner = PlannerAgent(
    dev_prompt=PLANNER_AGENT_PROMPT,
    llm=llm,
    messages=[],
    tools=tools,
    model_name="gpt-4.1-mini",
)
logger.info("Successfully initialized PlannerAgent")

In [20]:
content = "write an essay on the culture impact of the internet"

In [21]:
plan = planner.plan(content)

In [22]:
f'PLAN TYPE: {type(plan)}'

"PLAN TYPE: <class 'openai.types.responses.parsed_response.ParsedResponse[Plan]'>"

In [187]:
# plan

In [25]:
plan_parsed: Plan = plan.output_parsed

In [26]:
f'PLAN PARSED: {plan_parsed}'

"PLAN PARSED: original_query='write an essay on the culture impact of the internet' description='Plan to write an essay on the cultural impact of the internet with structured research, writing, and review tasks.' tasks=[PlannerTask(id=1, description='Research the cultural impact of the internet to gather relevant and accurate information for the essay.', tool_calls=[ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(keys=['query', 'sentences'], values=['cultural impact of the internet', '5']))], status='todo'), PlannerTask(id=2, description='Write the introduction paragraph for the essay, outlining the main topic and its significance.', tool_calls=[ToolCall(id='2', name='functions.wiki_search', arguments=ToolArguments(keys=['query'], values=['introduction to cultural impact of the internet']))], status='todo'), PlannerTask(id=3, description='Write the first body paragraph discussing the positive cultural impacts of the internet.', tool_calls=[ToolCall(id='3', name='

In [190]:
plan_parsed.tasks[0] # select first task

PlannerTask(id=1, description='Research the cultural impact of the internet to gather relevant facts, examples, and perspectives.', tool_calls=[ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(keys=['query', 'sentences'], values=['cultural impact of the internet', '5']))], status='todo')

In [191]:
plan_parsed.tasks[0].tool_calls[0] # select first tool call in task

ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(keys=['query', 'sentences'], values=['cultural impact of the internet', '5']))

In [193]:
plan_parsed.tasks[0].tool_calls[0].arguments # select the arguments in the frist tool call within the first task

ToolArguments(keys=['query', 'sentences'], values=['cultural impact of the internet', '5'])

In [194]:
plan_parsed.tasks[0].tool_calls[0].arguments.keys

['query', 'sentences']

In [207]:
tool = {'name': plan_parsed.tasks[0].tool_calls[0].name.split('.')[-1]}

In [208]:
tool_call_keys = plan_parsed.tasks[0].tool_calls[0].arguments.keys
tool_call_values = plan_parsed.tasks[0].tool_calls[0].arguments.values
for i in range(len(plan_parsed.tasks[0].tool_calls[0].arguments.keys)):
    tool[plan_parsed.tasks[0].tool_calls[0].arguments.keys[i]] = plan_parsed.tasks[0].tool_calls[0].arguments.values[i]

In [209]:
tool

{'name': 'wiki_search',
 'query': 'cultural impact of the internet',
 'sentences': '5'}

In [59]:
executor.tool_call_history

[]

In [68]:
res = await executor.execute_plan(plan_parsed)


              Plan to execute is: 

              Plan ID: write an essay on the culture impact of the internet
              Plan Description: Plan to write an essay on the cultural impact of the internet with structured research, writing, and review tasks.
         

              Executing task: 

              Task ID: 1 

              Task description: Research the cultural impact of the internet to gather relevant and accurate information for the essay. 

              Tool calls: [ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(keys=['query', 'sentences'], values=['cultural impact of the internet', '5']))] 

              Task status: todo 

              Previous Task Results: ['no context yet']
        
CALLING TOOLS: [{'name': 'wiki_search', 'arguments': {'query': 'cultural impact of the internet', 'sentences': '5'}}]
TOOL_CALLS: [{'name': 'wiki_search', 'arguments': {'query': 'cultural impact of the internet', 'sentences': '5'}}]
NAME: wiki_search
AR

KeyError: 'result'

In [38]:
f'RES TYPE: {type(res)}'

"RES TYPE: <class 'openai.types.responses.parsed_response.ParsedResponse[ToolCalls]'>"

In [72]:
res

['no context yet',
 {'task_id': 1,
  'task': 'Research the cultural impact of the internet, gathering information on key aspects such as communication, social behavior, access to information, and changes in cultural norms.',
  'results': [{'name': 'wiki_search',
    'arguments': {'query': 'cultural impact of the internet', 'retries': 2},
    'result': 'hello'}]},
 {'task_id': 2,
  'task': "Write the introduction paragraph that introduces the topic of the internet's cultural impact and outlines the main points to be discussed.",
  'results': [{'name': 'wiki_search',
    'arguments': {'query': 'internet cultural impact introduction',
     'retries': 1},
    'result': 'hello'}]},
 {'task_id': 3,
  'task': 'Write the first body paragraph discussing how the internet has transformed communication and social interactions in culture.',
  'results': [{'name': 'wiki_search',
    'arguments': {'query': 'internet impact on communication and social interaction',
     'retries': 2},
    'result': 'h

In [40]:
dir(res)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model_

In [73]:
res.output

[ParsedResponseOutputMessage[ToolCalls](id='msg_688a9e76085481a3bde383437260ae670d566afc8ad6c133', content=[ParsedResponseOutputText[ToolCalls](annotations=[], text='{"id":1,"tool_calls":[{"id":"1","name":"functions.wiki_search","arguments":{"query":"cultural impact of the internet","retries":2}}]}', type='output_text', logprobs=[], parsed=ToolCalls(id=1, tool_calls=[ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(query='cultural impact of the internet', retries=2))]))], role='assistant', status='completed', type='message'),
 ParsedResponseOutputMessage[ToolCalls](id='msg_688a9e76b78881a39e45b71462fea8580d566afc8ad6c133', content=[ParsedResponseOutputText[ToolCalls](annotations=[], text='{"id":2,"tool_calls":[{"id":"2","name":"functions.wiki_search","arguments":{"query":"cultural impact of the internet","retries":2}}]}', type='output_text', logprobs=[], parsed=ToolCalls(id=2, tool_calls=[ToolCall(id='2', name='functions.wiki_search', arguments=ToolArguments(query

In [77]:
res.output_parsed.tool_calls[0].name

'functions.wiki_search'

In [81]:
res.output_parsed.tool_calls[0].arguments.model_dump()

{'query': 'cultural impact of the internet', 'retries': 2}

In [78]:
res.output_parsed.tool_calls[0].name.split('.')[-1]

'wiki_search'

In [36]:
f'RES[0] TYPE: {type(res.output)}'

AttributeError: 'ParsedResponseOutputMessage[ToolCalls]' object has no attribute 'output'

In [ ]:
res[0]

In [ ]:
dir(res[0])

In [ ]:
res[0].output[1].name

In [ ]:
res[0].output[1].arguments

In [ ]:
res[0].output[0].name

In [ ]:
res[0].output[1].arguments

In [ ]:
for i in range(len(res[0].output)):
    tool_call = res[0].output[i]
    print(tool_call.name)
    print(tool_call.arguments)
    print()

In [ ]:
f'EXCUTE TASK RESULT: {res}'

In [ ]:
# TODO: have the tool_calls be in each plan
# TODO: have excutor take plan and execute the task